# F1 2022 Race winner predictions

# Importing Important Libraries

In [117]:
# Basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# To ignore warnings thrown in the notebook
import warnings
warnings.filterwarnings('ignore')
# Sklearn Libraries
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
# Library to see the epoch bar
from tqdm import tqdm
# Tensorflow libraries
import tensorflow as tf
from tensorflow import keras
from keras import backend as K

Wanted to involve weather data too but the one i fetched from the API and these datasets from kaggle did not match had no way to merge it with them so scrapped that plan

In [118]:
# Reading datasets we will be using
races = pd.read_csv('races.csv')
results = pd.read_csv('results.csv')
qualifying = pd.read_csv('qualifying.csv')
drivers = pd.read_csv('drivers.csv')
status = pd.read_csv('status.csv')
constructors = pd.read_csv('constructors.csv')
lap_times = pd.read_csv('lap_times.csv')

# Read and merge the data
Here we will do some early pre processing to make sure we can join these datasets to form one final dataframe

In [119]:
# Drop unecessary columns not required
results.drop(['resultId','number', 'positionText', 'positionOrder', 'laps', 'milliseconds', 'fastestLap', 'rank',
              'fastestLapTime', 'fastestLapSpeed','time','points'], axis=1, inplace=True)
# Rename columns for better understanding
results.rename(columns={'grid':'Quali_position','position':'Race_rank'}, inplace=True)
results.head()

,raceId,driverId,constructorId,Quali_position,Race_rank,statusId
0,18,1,1,1,1,1
1,18,2,2,5,2,1
2,18,3,3,7,3,1
3,18,4,4,11,4,1
4,18,5,1,3,5,1


In [120]:
# Drop unecessary columns not required
races.drop(['url', 'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time', 'quali_date',
            'quali_time', 'sprint_date', 'sprint_time','time'], axis=1, inplace=True)
# Rename columns for better understanding
races.rename(columns={'date':'race_date','name':'Circuit'}, inplace=True)
races.head()

,raceId,year,round,circuitId,Circuit,race_date
0,1,2009,1,1,Australian Grand Prix,2009-03-29
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05
2,3,2009,3,17,Chinese Grand Prix,2009-04-19
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26
4,5,2009,5,4,Spanish Grand Prix,2009-05-10


In [121]:
# Merge races and results datasets
df1 = pd.merge(races, results, on='raceId')
df1

,raceId,year,round,circuitId,Circuit,race_date,driverId,constructorId,Quali_position,Race_rank,statusId
0,1,2009,1,1,Australian Grand Prix,2009-03-29,18,23,1,1,1
1,1,2009,1,1,Australian Grand Prix,2009-03-29,22,23,2,2,1
2,1,2009,1,1,Australian Grand Prix,2009-03-29,15,7,20,3,1
3,1,2009,1,1,Australian Grand Prix,2009-03-29,10,7,19,4,1
4,1,2009,1,1,Australian Grand Prix,2009-03-29,4,4,10,5,1
...,...,...,...,...,...,...,...,...,...,...,...
26514,1132,2024,12,9,British Grand Prix,2024-07-07,839,214,18,16,12
26515,1132,2024,12,9,British Grand Prix,2024-07-07,815,9,0,17,12
26516,1132,2024,12,9,British Grand Prix,2024-07-07,855,15,14,18,12
26517,1132,2024,12,9,British Grand Prix,2024-07-07,847,131,1,\N,34


In [122]:
# Dropping unecessary columns
qualifying.drop(['qualifyId', 'number', 'position','constructorId'], axis=1, inplace=True)
qualifying.head()

,raceId,driverId,q1,q2,q3
0,18,1,1:26.572,1:25.187,1:26.714
1,18,9,1:26.103,1:25.315,1:26.869
2,18,5,1:25.664,1:25.452,1:27.079
3,18,13,1:25.994,1:25.691,1:27.178
4,18,2,1:25.960,1:25.518,1:27.236


In [123]:
# Merfing df1 along with qualiying data
df2 = pd.merge(df1, qualifying, how='left', on=['raceId','driverId'])
df2


,raceId,year,round,circuitId,Circuit,race_date,driverId,constructorId,Quali_position,Race_rank,statusId,q1,q2,q3
0,1,2009,1,1,Australian Grand Prix,2009-03-29,18,23,1,1,1,1:25.211,1:24.855,1:26.202
1,1,2009,1,1,Australian Grand Prix,2009-03-29,22,23,2,2,1,1:25.006,1:24.783,1:26.505
2,1,2009,1,1,Australian Grand Prix,2009-03-29,15,7,20,3,1,1:26.194,1:25.265,1:27.127
3,1,2009,1,1,Australian Grand Prix,2009-03-29,10,7,19,4,1,1:25.499,1:25.281,1:26.975
4,1,2009,1,1,Australian Grand Prix,2009-03-29,4,4,10,5,1,1:26.026,1:25.605,\N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26514,1132,2024,12,9,British Grand Prix,2024-07-07,839,214,18,16,12,1:34.557,\N,\N
26515,1132,2024,12,9,British Grand Prix,2024-07-07,815,9,0,17,12,1:38.348,\N,\N
26516,1132,2024,12,9,British Grand Prix,2024-07-07,855,15,14,18,12,1:31.190,1:27.867,\N
26517,1132,2024,12,9,British Grand Prix,2024-07-07,847,131,1,\N,34,1:30.106,1:26.723,1:25.819


In [124]:
# Rename columns for better understanding
drivers.rename(columns={'nationality':'Driver_nationality'}, inplace=True)

# Drop unnecessary columns
drivers.drop(['url','code','driverRef','number'], axis=1, inplace=True)
drivers.head()

,driverId,forename,surname,dob,Driver_nationality
0,1,Lewis,Hamilton,1985-01-07,British
1,2,Nick,Heidfeld,1977-05-10,German
2,3,Nico,Rosberg,1985-06-27,German
3,4,Fernando,Alonso,1981-07-29,Spanish
4,5,Heikki,Kovalainen,1981-10-19,Finnish


In [125]:
# Merge df2 with drivers dataset
df3 = pd.merge(df2, drivers, how='left', on=[ 'driverId'])
df3

,raceId,year,round,circuitId,Circuit,race_date,driverId,constructorId,Quali_position,Race_rank,statusId,q1,q2,q3,forename,surname,dob,Driver_nationality
0,1,2009,1,1,Australian Grand Prix,2009-03-29,18,23,1,1,1,1:25.211,1:24.855,1:26.202,Jenson,Button,1980-01-19,British
1,1,2009,1,1,Australian Grand Prix,2009-03-29,22,23,2,2,1,1:25.006,1:24.783,1:26.505,Rubens,Barrichello,1972-05-23,Brazilian
2,1,2009,1,1,Australian Grand Prix,2009-03-29,15,7,20,3,1,1:26.194,1:25.265,1:27.127,Jarno,Trulli,1974-07-13,Italian
3,1,2009,1,1,Australian Grand Prix,2009-03-29,10,7,19,4,1,1:25.499,1:25.281,1:26.975,Timo,Glock,1982-03-18,German
4,1,2009,1,1,Australian Grand Prix,2009-03-29,4,4,10,5,1,1:26.026,1:25.605,\N,Fernando,Alonso,1981-07-29,Spanish
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26514,1132,2024,12,9,British Grand Prix,2024-07-07,839,214,18,16,12,1:34.557,\N,\N,Esteban,Ocon,1996-09-17,French
26515,1132,2024,12,9,British Grand Prix,2024-07-07,815,9,0,17,12,1:38.348,\N,\N,Sergio,Pérez,1990-01-26,Mexican
26516,1132,2024,12,9,British Grand Prix,2024-07-07,855,15,14,18,12,1:31.190,1:27.867,\N,Guanyu,Zhou,1999-05-30,Chinese
26517,1132,2024,12,9,British Grand Prix,2024-07-07,847,131,1,\N,34,1:30.106,1:26.723,1:25.819,George,Russell,1998-02-15,British


In [126]:
# Renaming columns for better understanding
constructors.rename(columns={'name':'Constructor_name'}, inplace=True)

# Drop unnecessary columns
constructors.drop(['url','constructorRef','nationality'], axis=1, inplace=True)
constructors.head()

,constructorId,Constructor_name
0,1,McLaren
1,2,BMW Sauber
2,3,Williams
3,4,Renault
4,5,Toro Rosso


In [127]:
# Merge df3 with constructors dataset
df4 = pd.merge(df3, constructors, how='left', on=[ 'constructorId'])
df4

,raceId,year,round,circuitId,Circuit,race_date,driverId,constructorId,Quali_position,Race_rank,statusId,q1,q2,q3,forename,surname,dob,Driver_nationality,Constructor_name
0,1,2009,1,1,Australian Grand Prix,2009-03-29,18,23,1,1,1,1:25.211,1:24.855,1:26.202,Jenson,Button,1980-01-19,British,Brawn
1,1,2009,1,1,Australian Grand Prix,2009-03-29,22,23,2,2,1,1:25.006,1:24.783,1:26.505,Rubens,Barrichello,1972-05-23,Brazilian,Brawn
2,1,2009,1,1,Australian Grand Prix,2009-03-29,15,7,20,3,1,1:26.194,1:25.265,1:27.127,Jarno,Trulli,1974-07-13,Italian,Toyota
3,1,2009,1,1,Australian Grand Prix,2009-03-29,10,7,19,4,1,1:25.499,1:25.281,1:26.975,Timo,Glock,1982-03-18,German,Toyota
4,1,2009,1,1,Australian Grand Prix,2009-03-29,4,4,10,5,1,1:26.026,1:25.605,\N,Fernando,Alonso,1981-07-29,Spanish,Renault
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26514,1132,2024,12,9,British Grand Prix,2024-07-07,839,214,18,16,12,1:34.557,\N,\N,Esteban,Ocon,1996-09-17,French,Alpine F1 Team
26515,1132,2024,12,9,British Grand Prix,2024-07-07,815,9,0,17,12,1:38.348,\N,\N,Sergio,Pérez,1990-01-26,Mexican,Red Bull
26516,1132,2024,12,9,British Grand Prix,2024-07-07,855,15,14,18,12,1:31.190,1:27.867,\N,Guanyu,Zhou,1999-05-30,Chinese,Sauber
26517,1132,2024,12,9,British Grand Prix,2024-07-07,847,131,1,\N,34,1:30.106,1:26.723,1:25.819,George,Russell,1998-02-15,British,Mercedes


In [128]:
# Merge df4 with status dataset
df5 = pd.merge(df4, status, how='left', on=[ 'statusId'])
df5

,raceId,year,round,circuitId,Circuit,race_date,driverId,constructorId,Quali_position,Race_rank,statusId,q1,q2,q3,forename,surname,dob,Driver_nationality,Constructor_name,status
0,1,2009,1,1,Australian Grand Prix,2009-03-29,18,23,1,1,1,1:25.211,1:24.855,1:26.202,Jenson,Button,1980-01-19,British,Brawn,Finished
1,1,2009,1,1,Australian Grand Prix,2009-03-29,22,23,2,2,1,1:25.006,1:24.783,1:26.505,Rubens,Barrichello,1972-05-23,Brazilian,Brawn,Finished
2,1,2009,1,1,Australian Grand Prix,2009-03-29,15,7,20,3,1,1:26.194,1:25.265,1:27.127,Jarno,Trulli,1974-07-13,Italian,Toyota,Finished
3,1,2009,1,1,Australian Grand Prix,2009-03-29,10,7,19,4,1,1:25.499,1:25.281,1:26.975,Timo,Glock,1982-03-18,German,Toyota,Finished
4,1,2009,1,1,Australian Grand Prix,2009-03-29,4,4,10,5,1,1:26.026,1:25.605,\N,Fernando,Alonso,1981-07-29,Spanish,Renault,Finished
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26514,1132,2024,12,9,British Grand Prix,2024-07-07,839,214,18,16,12,1:34.557,\N,\N,Esteban,Ocon,1996-09-17,French,Alpine F1 Team,+2 Laps
26515,1132,2024,12,9,British Grand Prix,2024-07-07,815,9,0,17,12,1:38.348,\N,\N,Sergio,Pérez,1990-01-26,Mexican,Red Bull,+2 Laps
26516,1132,2024,12,9,British Grand Prix,2024-07-07,855,15,14,18,12,1:31.190,1:27.867,\N,Guanyu,Zhou,1999-05-30,Chinese,Sauber,+2 Laps
26517,1132,2024,12,9,British Grand Prix,2024-07-07,847,131,1,\N,34,1:30.106,1:26.723,1:25.819,George,Russell,1998-02-15,British,Mercedes,Water pressure


In [129]:
# Reading data of lap times
lap_times

,raceId,driverId,lap,position,time,milliseconds
0,841,20,1,1,1:38.109,98109
1,841,20,2,1,1:33.006,93006
2,841,20,3,1,1:32.713,92713
3,841,20,4,1,1:32.803,92803
4,841,20,5,1,1:32.342,92342
...,...,...,...,...,...,...
575024,1131,858,65,19,1:10.742,70742
575025,1131,858,66,19,1:10.855,70855
575026,1131,858,67,19,1:12.454,72454
575027,1131,858,68,19,1:13.607,73607


In [130]:
# Create a new df which will contain total lap time of that driver for a particular race
df_lap = pd.DataFrame(lap_times)

# Group by raceId and driverId and sum the lap times in milliseconds
total_lap_times = df_lap.groupby(["raceId", "driverId"])["milliseconds"].sum().reset_index()

# Updated function to include hours
def format_time(ms):
    seconds, milliseconds = divmod(ms, 1000)
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    return f"{hours}:{minutes:02d}:{seconds:02d}.{milliseconds:03d}"

total_lap_times["formatted_time"] = total_lap_times["milliseconds"].apply(format_time)

In [131]:
# Merge the new df created with df5
df = pd.merge(df5, total_lap_times, how='left', on=['raceId','driverId'])
df

,raceId,year,round,circuitId,Circuit,race_date,driverId,constructorId,Quali_position,Race_rank,...,q2,q3,forename,surname,dob,Driver_nationality,Constructor_name,status,milliseconds,formatted_time
0,1,2009,1,1,Australian Grand Prix,2009-03-29,18,23,1,1,...,1:24.855,1:26.202,Jenson,Button,1980-01-19,British,Brawn,Finished,5655784.0,1:34:15.784
1,1,2009,1,1,Australian Grand Prix,2009-03-29,22,23,2,2,...,1:24.783,1:26.505,Rubens,Barrichello,1972-05-23,Brazilian,Brawn,Finished,5656591.0,1:34:16.591
2,1,2009,1,1,Australian Grand Prix,2009-03-29,15,7,20,3,...,1:25.265,1:27.127,Jarno,Trulli,1974-07-13,Italian,Toyota,Finished,5657388.0,1:34:17.388
3,1,2009,1,1,Australian Grand Prix,2009-03-29,10,7,19,4,...,1:25.281,1:26.975,Timo,Glock,1982-03-18,German,Toyota,Finished,5660219.0,1:34:20.219
4,1,2009,1,1,Australian Grand Prix,2009-03-29,4,4,10,5,...,1:25.605,\N,Fernando,Alonso,1981-07-29,Spanish,Renault,Finished,5660663.0,1:34:20.663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26514,1132,2024,12,9,British Grand Prix,2024-07-07,839,214,18,16,...,\N,\N,Esteban,Ocon,1996-09-17,French,Alpine F1 Team,+2 Laps,NaN,NaN
26515,1132,2024,12,9,British Grand Prix,2024-07-07,815,9,0,17,...,\N,\N,Sergio,Pérez,1990-01-26,Mexican,Red Bull,+2 Laps,NaN,NaN
26516,1132,2024,12,9,British Grand Prix,2024-07-07,855,15,14,18,...,1:27.867,\N,Guanyu,Zhou,1999-05-30,Chinese,Sauber,+2 Laps,NaN,NaN
26517,1132,2024,12,9,British Grand Prix,2024-07-07,847,131,1,\N,...,1:26.723,1:25.819,George,Russell,1998-02-15,British,Mercedes,Water pressure,NaN,NaN


 Since the qualifying format was different in the olden days as compared to now which is of 3 rounds we will take data from 1996 - 2000 & 2006 - 2022.
 There is a gap in between beacuse for few years in between the qualifying format was different than 3 round qualifying so in order to make our model perfrom better we will take data in this way.

In [132]:
df= df[(df['year'].between(1996, 2000)) | (df['year'].between(2006, 2022))]
df

,raceId,year,round,circuitId,Circuit,race_date,driverId,constructorId,Quali_position,Race_rank,...,q2,q3,forename,surname,dob,Driver_nationality,Constructor_name,status,milliseconds,formatted_time
0,1,2009,1,1,Australian Grand Prix,2009-03-29,18,23,1,1,...,1:24.855,1:26.202,Jenson,Button,1980-01-19,British,Brawn,Finished,5655784.0,1:34:15.784
1,1,2009,1,1,Australian Grand Prix,2009-03-29,22,23,2,2,...,1:24.783,1:26.505,Rubens,Barrichello,1972-05-23,Brazilian,Brawn,Finished,5656591.0,1:34:16.591
2,1,2009,1,1,Australian Grand Prix,2009-03-29,15,7,20,3,...,1:25.265,1:27.127,Jarno,Trulli,1974-07-13,Italian,Toyota,Finished,5657388.0,1:34:17.388
3,1,2009,1,1,Australian Grand Prix,2009-03-29,10,7,19,4,...,1:25.281,1:26.975,Timo,Glock,1982-03-18,German,Toyota,Finished,5660219.0,1:34:20.219
4,1,2009,1,1,Australian Grand Prix,2009-03-29,4,4,10,5,...,1:25.605,\N,Fernando,Alonso,1981-07-29,Spanish,Renault,Finished,5660663.0,1:34:20.663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25835,1096,2022,22,24,Abu Dhabi Grand Prix,2022-11-20,854,210,12,16,...,1:25.225,\N,Mick,Schumacher,1999-03-22,German,Haas F1 Team,+1 Lap,5304984.0,1:28:24.984
25836,1096,2022,22,24,Abu Dhabi Grand Prix,2022-11-20,825,210,16,17,...,\N,\N,Kevin,Magnussen,1992-10-05,Danish,Haas F1 Team,+1 Lap,5316642.0,1:28:36.642
25837,1096,2022,22,24,Abu Dhabi Grand Prix,2022-11-20,1,131,5,18,...,1:24.774,1:24.508,Lewis,Hamilton,1985-01-07,British,Mercedes,Hydraulics,5056987.0,1:24:16.987
25838,1096,2022,22,24,Abu Dhabi Grand Prix,2022-11-20,849,3,20,19,...,\N,\N,Nicholas,Latifi,1995-06-29,Canadian,Williams,Collision damage,5195988.0,1:26:35.988


# Split the dataset into two parts so the testing dataset remains unseen while training

In [133]:
# Split the dataset into train and test sets
df_train = df[(df['year'].between(1996, 2000)) | (df['year'].between(2006, 2021))]
df_test = df[df['year'] == 2022]

# Verify the splits
print(f"Training data (years 1996-2000, 2006-2021): {df_train.shape}")
print(f"Testing data (year 2022): {df_test.shape}")


Training data (years 1996-2000, 2006-2021): (8345, 22)
Testing data (year 2022): (440, 22)


In [134]:
df_train

,raceId,year,round,circuitId,Circuit,race_date,driverId,constructorId,Quali_position,Race_rank,...,q2,q3,forename,surname,dob,Driver_nationality,Constructor_name,status,milliseconds,formatted_time
0,1,2009,1,1,Australian Grand Prix,2009-03-29,18,23,1,1,...,1:24.855,1:26.202,Jenson,Button,1980-01-19,British,Brawn,Finished,5655784.0,1:34:15.784
1,1,2009,1,1,Australian Grand Prix,2009-03-29,22,23,2,2,...,1:24.783,1:26.505,Rubens,Barrichello,1972-05-23,Brazilian,Brawn,Finished,5656591.0,1:34:16.591
2,1,2009,1,1,Australian Grand Prix,2009-03-29,15,7,20,3,...,1:25.265,1:27.127,Jarno,Trulli,1974-07-13,Italian,Toyota,Finished,5657388.0,1:34:17.388
3,1,2009,1,1,Australian Grand Prix,2009-03-29,10,7,19,4,...,1:25.281,1:26.975,Timo,Glock,1982-03-18,German,Toyota,Finished,5660219.0,1:34:20.219
4,1,2009,1,1,Australian Grand Prix,2009-03-29,4,4,10,5,...,1:25.605,\N,Fernando,Alonso,1981-07-29,Spanish,Renault,Finished,5660663.0,1:34:20.663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25415,1073,2021,22,24,Abu Dhabi Grand Prix,2021-12-12,849,3,16,\N,...,\N,\N,Nicholas,Latifi,1995-06-29,Canadian,Williams,Accident,4615491.0,1:16:55.491
25416,1073,2021,22,24,Abu Dhabi Grand Prix,2021-12-12,841,51,14,\N,...,1:24.251,\N,Antonio,Giovinazzi,1993-12-14,Italian,Alfa Romeo,Gearbox,3020298.0,0:50:20.298
25417,1073,2021,22,24,Abu Dhabi Grand Prix,2021-12-12,847,3,17,\N,...,\N,\N,George,Russell,1998-02-15,British,Williams,Gearbox,2470855.0,0:41:10.855
25418,1073,2021,22,24,Abu Dhabi Grand Prix,2021-12-12,8,51,18,\N,...,\N,\N,Kimi,Räikkönen,1979-10-17,Finnish,Alfa Romeo,Brakes,2363611.0,0:39:23.611


In [135]:
df_test

,raceId,year,round,circuitId,Circuit,race_date,driverId,constructorId,Quali_position,Race_rank,...,q2,q3,forename,surname,dob,Driver_nationality,Constructor_name,status,milliseconds,formatted_time
24980,1074,2022,1,3,Bahrain Grand Prix,2022-03-20,844,6,1,1,...,1:30.932,1:30.558,Charles,Leclerc,1997-10-16,Monegasque,Ferrari,Finished,5853584.0,1:37:33.584
24981,1074,2022,1,3,Bahrain Grand Prix,2022-03-20,832,6,3,2,...,1:30.787,1:30.687,Carlos,Sainz,1994-09-01,Spanish,Ferrari,Finished,5859182.0,1:37:39.182
24982,1074,2022,1,3,Bahrain Grand Prix,2022-03-20,1,131,5,3,...,1:31.048,1:31.238,Lewis,Hamilton,1985-01-07,British,Mercedes,Finished,5863259.0,1:37:43.259
24983,1074,2022,1,3,Bahrain Grand Prix,2022-03-20,847,131,9,4,...,1:31.252,1:32.216,George,Russell,1998-02-15,British,Mercedes,Finished,5864795.0,1:37:44.795
24984,1074,2022,1,3,Bahrain Grand Prix,2022-03-20,825,210,7,5,...,1:31.461,1:31.808,Kevin,Magnussen,1992-10-05,Danish,Haas F1 Team,Finished,5868338.0,1:37:48.338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25835,1096,2022,22,24,Abu Dhabi Grand Prix,2022-11-20,854,210,12,16,...,1:25.225,\N,Mick,Schumacher,1999-03-22,German,Haas F1 Team,+1 Lap,5304984.0,1:28:24.984
25836,1096,2022,22,24,Abu Dhabi Grand Prix,2022-11-20,825,210,16,17,...,\N,\N,Kevin,Magnussen,1992-10-05,Danish,Haas F1 Team,+1 Lap,5316642.0,1:28:36.642
25837,1096,2022,22,24,Abu Dhabi Grand Prix,2022-11-20,1,131,5,18,...,1:24.774,1:24.508,Lewis,Hamilton,1985-01-07,British,Mercedes,Hydraulics,5056987.0,1:24:16.987
25838,1096,2022,22,24,Abu Dhabi Grand Prix,2022-11-20,849,3,20,19,...,\N,\N,Nicholas,Latifi,1995-06-29,Canadian,Williams,Collision damage,5195988.0,1:26:35.988


# Data Pre processing and Feature Engineering

In [136]:
# Join the datasets again for pre processing
df = pd.concat([df_train, df_test])
df

,raceId,year,round,circuitId,Circuit,race_date,driverId,constructorId,Quali_position,Race_rank,...,q2,q3,forename,surname,dob,Driver_nationality,Constructor_name,status,milliseconds,formatted_time
0,1,2009,1,1,Australian Grand Prix,2009-03-29,18,23,1,1,...,1:24.855,1:26.202,Jenson,Button,1980-01-19,British,Brawn,Finished,5655784.0,1:34:15.784
1,1,2009,1,1,Australian Grand Prix,2009-03-29,22,23,2,2,...,1:24.783,1:26.505,Rubens,Barrichello,1972-05-23,Brazilian,Brawn,Finished,5656591.0,1:34:16.591
2,1,2009,1,1,Australian Grand Prix,2009-03-29,15,7,20,3,...,1:25.265,1:27.127,Jarno,Trulli,1974-07-13,Italian,Toyota,Finished,5657388.0,1:34:17.388
3,1,2009,1,1,Australian Grand Prix,2009-03-29,10,7,19,4,...,1:25.281,1:26.975,Timo,Glock,1982-03-18,German,Toyota,Finished,5660219.0,1:34:20.219
4,1,2009,1,1,Australian Grand Prix,2009-03-29,4,4,10,5,...,1:25.605,\N,Fernando,Alonso,1981-07-29,Spanish,Renault,Finished,5660663.0,1:34:20.663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25835,1096,2022,22,24,Abu Dhabi Grand Prix,2022-11-20,854,210,12,16,...,1:25.225,\N,Mick,Schumacher,1999-03-22,German,Haas F1 Team,+1 Lap,5304984.0,1:28:24.984
25836,1096,2022,22,24,Abu Dhabi Grand Prix,2022-11-20,825,210,16,17,...,\N,\N,Kevin,Magnussen,1992-10-05,Danish,Haas F1 Team,+1 Lap,5316642.0,1:28:36.642
25837,1096,2022,22,24,Abu Dhabi Grand Prix,2022-11-20,1,131,5,18,...,1:24.774,1:24.508,Lewis,Hamilton,1985-01-07,British,Mercedes,Hydraulics,5056987.0,1:24:16.987
25838,1096,2022,22,24,Abu Dhabi Grand Prix,2022-11-20,849,3,20,19,...,\N,\N,Nicholas,Latifi,1995-06-29,Canadian,Williams,Collision damage,5195988.0,1:26:35.988


## DidNotFinish

We create this column so our model can understand which drivers finish the race and which didnt


In [137]:
# List of statuses indicating a driver did not finish the race
non_finish_statuses = ['Collision', 'Differential', 'Suspension',
       'Spun off', 'Accident', 'Disqualified', 'Engine', 'Retired', 'Electronics', 'Oil pressure', 'Hydraulics', 'Gearbox',
       'Brakes', 'Wheel', 'Overheating', 'Withdrew','Transmission', 'Oil leak', 'Clutch', 'Injured', 'Out of fuel',
       'Electrical', 'Radiator','Mechanical', 'Tyre',
       'Driver Seat', 'Puncture', 'Driveshaft', 'Fuel pressure',
       'Front wing', 'Water pressure', 'Refuelling', 'Throttle',
       'Steering', 'Technical', 'Broken wing', 'Heat shield fire',
       'Exhaust','Wheel rim', 'Water leak', 'Fuel pump',
       'Track rod','Pneumatics','Engine fire', 'Tyre puncture', 'Wheel nut',
       'Not classified','Handling', 'Rear wing',
       'Fire', 'Fuel system', 'Oil line', 'Fuel rig', 'Launch control',
       'Fuel', 'Power loss', '107% Rule', 'Safety', 'Drivetrain',
       'Ignition', 'Did not qualify', 'Injury', 'Chassis', 'Battery',
       'Stalled', 'Halfshaft', 'Crankshaft', 'Safety concerns',
       'Not restarted', 'Alternator', 'Wheel bearing', 'Physical',
       'Vibrations', 'Underweight', 'Safety belt', 'Oil pump',
       'Fuel leak', 'Did not prequalify', 'Excluded', 'Injection',
       'Distributor','Driver unwell', 'Turbo',
       'CV joint', 'Water pump', 'Fatal accident', 'Spark plugs',
       'Fuel pipe', 'Eye injury', 'Oil pipe', 'Axle', 'Water pipe', 'Supercharger', 'Engine misfire', 'Collision damage', 'ERS', 'Power Unit',
       'Brake duct', 'Seat', 'Damage', 'Debris', 'Illness',
       'Cooling system', 'Undertray']

# Create a binary column: 1 for not finishing, 0 for finishing
df['DidNotFinish'] = np.where(df['status'].isin(non_finish_statuses), 1, 0)

## Target Variable "IsRaceWinner"


In [138]:
# Convert the column to numeric
df['Race_rank'] = pd.to_numeric(df['Race_rank'], errors='coerce')

# Create target variable
df['IsRaceWinner'] = df['Race_rank'].apply(lambda x: 1 if x == 1 else 0)

## Driver name


In [139]:
# Join two column to see full driver name
df['Driver_name'] = df['forename'] + ' ' + df['surname']
df.drop(['forename','surname'], axis=1, inplace=True)

## Replacing Null Values in milliseconds

In the dataset there is laptime even for drivers who have not completed the race . This is false information so we will give it a flagship value so the model knows

In [140]:
# Replace '\N' with NaN using a raw string
df = df.replace(r'\N', np.nan)

# Assign flagship value for drivers who did not finish the race
df.loc[df['DidNotFinish'] == 1, 'milliseconds'] = 999

In [141]:
# Rename column for better understanding
df.rename(columns={'milliseconds' : 'LapTime'},inplace=True)
df

,raceId,year,round,circuitId,Circuit,race_date,driverId,constructorId,Quali_position,Race_rank,...,q3,dob,Driver_nationality,Constructor_name,status,LapTime,formatted_time,DidNotFinish,IsRaceWinner,Driver_name
0,1,2009,1,1,Australian Grand Prix,2009-03-29,18,23,1,1.0,...,1:26.202,1980-01-19,British,Brawn,Finished,5655784.0,1:34:15.784,0,1,Jenson Button
1,1,2009,1,1,Australian Grand Prix,2009-03-29,22,23,2,2.0,...,1:26.505,1972-05-23,Brazilian,Brawn,Finished,5656591.0,1:34:16.591,0,0,Rubens Barrichello
2,1,2009,1,1,Australian Grand Prix,2009-03-29,15,7,20,3.0,...,1:27.127,1974-07-13,Italian,Toyota,Finished,5657388.0,1:34:17.388,0,0,Jarno Trulli
3,1,2009,1,1,Australian Grand Prix,2009-03-29,10,7,19,4.0,...,1:26.975,1982-03-18,German,Toyota,Finished,5660219.0,1:34:20.219,0,0,Timo Glock
4,1,2009,1,1,Australian Grand Prix,2009-03-29,4,4,10,5.0,...,NaN,1981-07-29,Spanish,Renault,Finished,5660663.0,1:34:20.663,0,0,Fernando Alonso
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25835,1096,2022,22,24,Abu Dhabi Grand Prix,2022-11-20,854,210,12,16.0,...,NaN,1999-03-22,German,Haas F1 Team,+1 Lap,5304984.0,1:28:24.984,0,0,Mick Schumacher
25836,1096,2022,22,24,Abu Dhabi Grand Prix,2022-11-20,825,210,16,17.0,...,NaN,1992-10-05,Danish,Haas F1 Team,+1 Lap,5316642.0,1:28:36.642,0,0,Kevin Magnussen
25837,1096,2022,22,24,Abu Dhabi Grand Prix,2022-11-20,1,131,5,18.0,...,1:24.508,1985-01-07,British,Mercedes,Hydraulics,999.0,1:24:16.987,1,0,Lewis Hamilton
25838,1096,2022,22,24,Abu Dhabi Grand Prix,2022-11-20,849,3,20,19.0,...,NaN,1995-06-29,Canadian,Williams,Collision damage,999.0,1:26:35.988,1,0,Nicholas Latifi


## Driver related past performace features


In [142]:
# Creating two features realted to past performance of driver
df['podium_last_5_races'] = (
    df.groupby('Driver_name')['Race_rank']
    .rolling(window=5, min_periods=1)
    .apply(lambda x: (x <= 3).sum(), raw=True)  # Check for podium (position <= 3)
    .reset_index(0, drop=True)
)


df['wins_last_5_races'] = (
    df.groupby('Driver_name')['Race_rank']
    .rolling(window=5, min_periods=1)
    .apply(lambda x: (x == 1).sum(), raw=True)  # Check for wins (position == 1)
    .reset_index(0, drop=True))

## Drive age

In [143]:
# Calculate driver age for that race
from dateutil.relativedelta import *
df['race_date'] = pd.to_datetime(df.race_date)
df['dob'] = pd.to_datetime(df.dob)
df['driver_age'] = df.apply(lambda x: relativedelta(x['race_date'], x['dob']).years, axis=1)

## Driver Experience



Experience matters in F1 as the cars are not easy to handle and we have seen its evrytime an experienced driver who has advantage over inexperienced and he is more likely to win . So we create a new feature

In [144]:
# Create a new feature called driver_experience
df['driver_experience'] = df.groupby('driverId')['year'].transform('nunique')
df['driver_experience'] = df['driver_experience'] * df['driver_age']

## Driver related Features


In [145]:
 #Calculate average position in the last 5 races for each driver
df['avg_pos_last5_race'] = df.groupby('driverId')['Race_rank'].transform(
    lambda x: x.rolling(5, min_periods=1).mean())

In [146]:
# Calculate career wins for each driver
df['career_wins'] = df.groupby('driverId')['IsRaceWinner'].transform(
    lambda x: (x == 1).cumsum())

## Constructor related feature


In [147]:
# Calculate constructor wins in the last 5 races
df['constructor_wins_last_5_races'] = df.groupby('constructorId')['IsRaceWinner'].transform(
    lambda x: (x == 1).rolling(5, min_periods=1).sum())

## Grid (Qualifying position) impact on racewin


In [148]:
# Calculate the grid position impact based on historical performance
# This could be calculated as the likelihood of winning from each grid position

# First, calculate the win rate for each grid position across all races
grid_win_rate = df[df['IsRaceWinner'] == 1].groupby('Quali_position').size() / df.groupby('Quali_position').size()

# Merge this data back into the main dataframe to get grid position impact
df['Quali_position_impact'] = df['Quali_position'].map(grid_win_rate)

## Driver and circuit history related feature


Sometimes drivers who have won constantly or are at their home gp they are more likely to win so we create a feature to see how many times the driver has won at that circuit

In [149]:
# Calculate driver-circuit win rate
driver_circuit_wins = df[df['IsRaceWinner'] == 1].groupby(['driverId', 'circuitId']).size()
driver_circuit_races = df.groupby(['driverId', 'circuitId']).size()

# Calculate win rate by dividing the number of wins by total races
driver_circuit_win_rate = driver_circuit_wins / driver_circuit_races

# Merge this back into the dataframe
df['driver_circuit_win_rate'] = df.apply(
    lambda row: driver_circuit_win_rate.get((row['driverId'], row['circuitId']), 0), axis=1)

In [150]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8785 entries, 0 to 25839
Data columns (total 32 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   raceId                         8785 non-null   int64         
 1   year                           8785 non-null   int64         
 2   round                          8785 non-null   int64         
 3   circuitId                      8785 non-null   int64         
 4   Circuit                        8785 non-null   object        
 5   race_date                      8785 non-null   datetime64[ns]
 6   driverId                       8785 non-null   int64         
 7   constructorId                  8785 non-null   int64         
 8   Quali_position                 8785 non-null   int64         
 9   Race_rank                      6713 non-null   float64       
 10  statusId                       8785 non-null   int64         
 11  q1                   

## Dropping columns

Originally thought of keeping driverId, circuitId so that we can take a particular circuit to predict the winner rather than just predicting all the races at once but couldnt quite figure it out.

Even thought of keeping q1, q2, q3 but ended up dropping since it had three columns there would be null plus already had Quali_position which basically is given to driver after qualifying. It felt redndant feature so removed it.

In [151]:
# These columns are not required so we will drop them
df.drop(['circuitId','driverId','statusId','race_date','constructorId','Race_rank','q1','q2','q3','dob',
         'status','formatted_time'],axis=1,inplace=True)
df

,raceId,year,round,Circuit,Quali_position,Driver_nationality,Constructor_name,LapTime,DidNotFinish,IsRaceWinner,Driver_name,podium_last_5_races,wins_last_5_races,driver_age,driver_experience,avg_pos_last5_race,career_wins,constructor_wins_last_5_races,Quali_position_impact,driver_circuit_win_rate
0,1,2009,1,Australian Grand Prix,1,British,Brawn,5655784.0,0,1,Jenson Button,1.0,1.0,29,377,1.00,1,1.0,0.481752,0.250000
1,1,2009,1,Australian Grand Prix,2,Brazilian,Brawn,5656591.0,0,0,Rubens Barrichello,1.0,0.0,36,396,2.00,0,1.0,0.270732,NaN
2,1,2009,1,Australian Grand Prix,20,Italian,Toyota,5657388.0,0,0,Jarno Trulli,1.0,0.0,34,340,3.00,0,0.0,NaN,NaN
3,1,2009,1,Australian Grand Prix,19,German,Toyota,5660219.0,0,0,Timo Glock,0.0,0.0,27,135,4.00,0,0.0,NaN,NaN
4,1,2009,1,Australian Grand Prix,10,Spanish,Renault,5660663.0,0,0,Fernando Alonso,0.0,0.0,27,405,5.00,0,0.0,0.009780,0.076923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25835,1096,2022,22,Abu Dhabi Grand Prix,12,German,Haas F1 Team,5304984.0,0,0,Mick Schumacher,0.0,0.0,23,46,15.40,0,0.0,NaN,NaN
25836,1096,2022,22,Abu Dhabi Grand Prix,16,Danish,Haas F1 Team,5316642.0,0,0,Kevin Magnussen,0.0,0.0,30,240,14.25,0,0.0,NaN,NaN
25837,1096,2022,22,Abu Dhabi Grand Prix,5,British,Mercedes,999.0,1,0,Lewis Hamilton,3.0,0.0,37,592,5.80,103,1.0,0.024331,0.357143
25838,1096,2022,22,Abu Dhabi Grand Prix,20,Canadian,Williams,999.0,1,0,Nicholas Latifi,0.0,0.0,27,81,15.80,0,0.0,NaN,NaN


## Dealing with Null Values

The Null values in feature engineered columns like



*   avg_pos_last_5_races - Suggest that at that time there are races where a driver has participated in fewer than five races, calculating the average position from the last five races would be impossible, leading to missing values.
*   driver_circuit_win_rate - This might mean that the driver has never won on that circuit so it is null
*   wins_last_5_races   -  This might mean that the driver has never won in last 5 races so it is null


*   Quali_position_impact - That particular qualifying postion never had a win in race so it is null

Instead of doing any mean median imputation it is in our best interest to fill them with 0 instead.



In [152]:
df.fillna(0, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8785 entries, 0 to 25839
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   raceId                         8785 non-null   int64  
 1   year                           8785 non-null   int64  
 2   round                          8785 non-null   int64  
 3   Circuit                        8785 non-null   object 
 4   Quali_position                 8785 non-null   int64  
 5   Driver_nationality             8785 non-null   object 
 6   Constructor_name               8785 non-null   object 
 7   LapTime                        8785 non-null   float64
 8   DidNotFinish                   8785 non-null   int64  
 9   IsRaceWinner                   8785 non-null   int64  
 10  Driver_name                    8785 non-null   object 
 11  podium_last_5_races            8785 non-null   float64
 12  wins_last_5_races              8785 non-null   float

## Create a df_eval to evaluate results at the end

In [97]:
df_eval = df[df['year'] == 2022]
df_eval

,raceId,year,round,Circuit,Quali_position,Driver_nationality,Constructor_name,LapTime,DidNotFinish,IsRaceWinner,Driver_name,podium_last_5_races,wins_last_5_races,driver_age,driver_experience,avg_pos_last5_race,career_wins,constructor_wins_last_5_races,Quali_position_impact,driver_circuit_win_rate
24980,1074,2022,1,Bahrain Grand Prix,1,Monegasque,Ferrari,5853584.0,0,1,Charles Leclerc,1.0,1.0,24,120,5.60,3,1.0,0.481752,0.166667
24981,1074,2022,1,Bahrain Grand Prix,3,Spanish,Ferrari,5859182.0,0,0,Carlos Sainz,2.0,0.0,27,216,5.00,0,1.0,0.121655,0.000000
24982,1074,2022,1,Bahrain Grand Prix,5,British,Mercedes,5863259.0,0,0,Lewis Hamilton,5.0,2.0,37,592,1.80,103,1.0,0.024331,0.333333
24983,1074,2022,1,Bahrain Grand Prix,9,British,Mercedes,5864795.0,0,0,George Russell,0.0,0.0,24,96,11.00,0,0.0,0.000000,0.000000
24984,1074,2022,1,Bahrain Grand Prix,7,Danish,Haas F1 Team,5868338.0,0,0,Kevin Magnussen,0.0,0.0,29,232,14.40,0,0.0,0.007317,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25835,1096,2022,22,Abu Dhabi Grand Prix,12,German,Haas F1 Team,5304984.0,0,0,Mick Schumacher,0.0,0.0,23,46,15.40,0,0.0,0.000000,0.000000
25836,1096,2022,22,Abu Dhabi Grand Prix,16,Danish,Haas F1 Team,5316642.0,0,0,Kevin Magnussen,0.0,0.0,30,240,14.25,0,0.0,0.000000,0.000000
25837,1096,2022,22,Abu Dhabi Grand Prix,5,British,Mercedes,999.0,1,0,Lewis Hamilton,3.0,0.0,37,592,5.80,103,1.0,0.024331,0.357143
25838,1096,2022,22,Abu Dhabi Grand Prix,20,Canadian,Williams,999.0,1,0,Nicholas Latifi,0.0,0.0,27,81,15.80,0,0.0,0.000000,0.000000


# One Hot Encoding and feature scaling

In [98]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Extracting Categorical Columns
categorical_data= df.select_dtypes(include=['object']).columns

# Applying one-hot encoding to these categorical columns
df = pd.get_dummies(df, columns=categorical_data)
df

,raceId,year,round,Quali_position,LapTime,DidNotFinish,IsRaceWinner,podium_last_5_races,wins_last_5_races,driver_age,...,Driver_name_Timo Glock,Driver_name_Toranosuke Takagi,Driver_name_Ukyo Katayama,Driver_name_Valtteri Bottas,Driver_name_Vincenzo Sospiri,Driver_name_Vitaly Petrov,Driver_name_Vitantonio Liuzzi,Driver_name_Will Stevens,Driver_name_Yuji Ide,Driver_name_Yuki Tsunoda
0,1,2009,1,1,5655784.0,0,1,1.0,1.0,29,...,False,False,False,False,False,False,False,False,False,False
1,1,2009,1,2,5656591.0,0,0,1.0,0.0,36,...,False,False,False,False,False,False,False,False,False,False
2,1,2009,1,20,5657388.0,0,0,1.0,0.0,34,...,False,False,False,False,False,False,False,False,False,False
3,1,2009,1,19,5660219.0,0,0,0.0,0.0,27,...,True,False,False,False,False,False,False,False,False,False
4,1,2009,1,10,5660663.0,0,0,0.0,0.0,27,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25835,1096,2022,22,12,5304984.0,0,0,0.0,0.0,23,...,False,False,False,False,False,False,False,False,False,False
25836,1096,2022,22,16,5316642.0,0,0,0.0,0.0,30,...,False,False,False,False,False,False,False,False,False,False
25837,1096,2022,22,5,999.0,1,0,3.0,0.0,37,...,False,False,False,False,False,False,False,False,False,False
25838,1096,2022,22,20,999.0,1,0,0.0,0.0,27,...,False,False,False,False,False,False,False,False,False,False


Tried keeping the one hot columns in boolean but the model was performing worse so converted them to int

In [99]:
# Converting to  1 and 0 for all boolean values in the DataFrame
df = df.applymap(lambda x: 1 if x == True else (0 if x == False else x))
df

,raceId,year,round,Quali_position,LapTime,DidNotFinish,IsRaceWinner,podium_last_5_races,wins_last_5_races,driver_age,...,Driver_name_Timo Glock,Driver_name_Toranosuke Takagi,Driver_name_Ukyo Katayama,Driver_name_Valtteri Bottas,Driver_name_Vincenzo Sospiri,Driver_name_Vitaly Petrov,Driver_name_Vitantonio Liuzzi,Driver_name_Will Stevens,Driver_name_Yuji Ide,Driver_name_Yuki Tsunoda
0,1,2009,1,1,5655784.0,0,1,1.0,1.0,29,...,0,0,0,0,0,0,0,0,0,0
1,1,2009,1,2,5656591.0,0,0,1.0,0.0,36,...,0,0,0,0,0,0,0,0,0,0
2,1,2009,1,20,5657388.0,0,0,1.0,0.0,34,...,0,0,0,0,0,0,0,0,0,0
3,1,2009,1,19,5660219.0,0,0,0.0,0.0,27,...,1,0,0,0,0,0,0,0,0,0
4,1,2009,1,10,5660663.0,0,0,0.0,0.0,27,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25835,1096,2022,22,12,5304984.0,0,0,0.0,0.0,23,...,0,0,0,0,0,0,0,0,0,0
25836,1096,2022,22,16,5316642.0,0,0,0.0,0.0,30,...,0,0,0,0,0,0,0,0,0,0
25837,1096,2022,22,5,999.0,1,0,3.0,0.0,37,...,0,0,0,0,0,0,0,0,0,0
25838,1096,2022,22,20,999.0,1,0,0.0,0.0,27,...,0,0,0,0,0,0,0,0,0,0


Tried also tried normalizing through Keras Preprocessing Library but it was thrwoing error during grid search so scrapped the idea and went with standard scaler

In [100]:
from sklearn.preprocessing import StandardScaler
# Numerical columns
numerical_columns = [
    'Quali_position', 'LapTime', 'podium_last_5_races', 'wins_last_5_races',
    'driver_age', 'driver_experience', 'avg_pos_last5_race', 'career_wins',
    'constructor_wins_last_5_races', 'Quali_position_impact',
    'driver_circuit_win_rate'
]

# Initialize the StandardScaler
scaler = StandardScaler()

# Scaling the numerical columns
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

df


,raceId,year,round,Quali_position,LapTime,DidNotFinish,IsRaceWinner,podium_last_5_races,wins_last_5_races,driver_age,...,Driver_name_Timo Glock,Driver_name_Toranosuke Takagi,Driver_name_Ukyo Katayama,Driver_name_Valtteri Bottas,Driver_name_Vincenzo Sospiri,Driver_name_Vitaly Petrov,Driver_name_Vitantonio Liuzzi,Driver_name_Will Stevens,Driver_name_Yuji Ide,Driver_name_Yuki Tsunoda
0,1,2009,1,-1.607995,0.504883,0,1,0.246286,1.131240,0.254747,...,0,0,0,0,0,0,0,0,0,0
1,1,2009,1,-1.448695,0.505186,0,0,0.246286,-0.341702,1.667852,...,0,0,0,0,0,0,0,0,0,0
2,1,2009,1,1.418703,0.505485,0,0,0.246286,-0.341702,1.264108,...,0,0,0,0,0,0,0,0,0,0
3,1,2009,1,1.259403,0.506548,0,0,-0.556841,-0.341702,-0.148997,...,1,0,0,0,0,0,0,0,0,0
4,1,2009,1,-0.174296,0.506715,0,0,-0.556841,-0.341702,-0.148997,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25835,1096,2022,22,0.144304,0.373129,0,0,-0.556841,-0.341702,-0.956486,...,0,0,0,0,0,0,0,0,0,0
25836,1096,2022,22,0.781503,0.377508,0,0,-0.556841,-0.341702,0.456619,...,0,0,0,0,0,0,0,0,0,0
25837,1096,2022,22,-0.970795,-1.618941,1,0,1.852542,-0.341702,1.869724,...,0,0,0,0,0,0,0,0,0,0
25838,1096,2022,22,1.418703,-1.618941,1,0,-0.556841,-0.341702,-0.148997,...,0,0,0,0,0,0,0,0,0,0


# Splitting the dataset back to training and testing


In [101]:
# Splitting the dataset back into training and testing
df_train = df.iloc[:8345]
df_test = df.iloc[8345:]
print(f'Training data shape : {df_train.shape}')
print(f'Test data shape : {df_test.shape}')

Training data shape : (8345, 256)
Test data shape : (440, 256)


In [102]:
# Sort the data by year since feeding it to training will be easier this way
df_train.sort_values(by=['year','round'], ascending=True, inplace=True)
df_train

,raceId,year,round,Quali_position,LapTime,DidNotFinish,IsRaceWinner,podium_last_5_races,wins_last_5_races,driver_age,...,Driver_name_Timo Glock,Driver_name_Toranosuke Takagi,Driver_name_Ukyo Katayama,Driver_name_Valtteri Bottas,Driver_name_Vincenzo Sospiri,Driver_name_Vitaly Petrov,Driver_name_Vitantonio Liuzzi,Driver_name_Will Stevens,Driver_name_Yuji Ide,Driver_name_Yuki Tsunoda
4720,224,1996,1,-1.448695,0.472848,0,1,0.246286,1.131240,1.465980,...,0,0,0,0,0,0,0,0,0,0
4721,224,1996,1,-1.607995,0.487128,0,0,2.655669,2.604182,-0.754613,...,0,0,0,0,0,0,0,0,0,0
4722,224,1996,1,-1.289395,0.496349,0,0,1.049414,-0.341702,0.456619,...,0,0,0,0,0,0,0,0,0,0
4723,224,1996,1,-0.652196,0.501782,0,0,-0.556841,-0.341702,1.667852,...,0,0,0,0,0,0,0,0,0,0
4724,224,1996,1,-0.970795,0.508555,0,0,0.246286,1.131240,-0.148997,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25415,1073,2021,22,0.781503,-1.618941,1,0,-0.556841,-0.341702,-0.350869,...,0,0,0,0,0,0,0,0,0,0
25416,1073,2021,22,0.462903,-1.618941,1,0,-0.556841,-0.341702,-0.148997,...,0,0,0,0,0,0,0,0,0,0
25417,1073,2021,22,0.940803,-1.618941,1,0,-0.556841,-0.341702,-0.956486,...,0,0,0,0,0,0,0,0,0,0
25418,1073,2021,22,1.100103,-1.618941,1,0,-0.556841,-0.341702,2.879085,...,0,0,0,0,0,0,0,0,0,0


In [104]:
# Sort the data by year and round since evaluating will be easier
df_test.sort_values(by=['year','round'], ascending=True, inplace=True)
df_test

,raceId,year,round,Quali_position,LapTime,DidNotFinish,IsRaceWinner,podium_last_5_races,wins_last_5_races,driver_age,...,Driver_name_Timo Glock,Driver_name_Toranosuke Takagi,Driver_name_Ukyo Katayama,Driver_name_Valtteri Bottas,Driver_name_Vincenzo Sospiri,Driver_name_Vitaly Petrov,Driver_name_Vitantonio Liuzzi,Driver_name_Will Stevens,Driver_name_Yuji Ide,Driver_name_Yuki Tsunoda
24980,1074,2022,1,-1.607995,0.579172,0,1,0.246286,1.131240,-0.754613,...,0,0,0,0,0,0,0,0,0,0
24981,1074,2022,1,-1.289395,0.581275,0,0,1.049414,-0.341702,-0.148997,...,0,0,0,0,0,0,0,0,0,0
24982,1074,2022,1,-0.970795,0.582806,0,0,3.458797,2.604182,1.869724,...,0,0,0,0,0,0,0,0,0,0
24983,1074,2022,1,-0.333596,0.583383,0,0,-0.556841,-0.341702,-0.754613,...,0,0,0,0,0,0,0,0,0,0
24984,1074,2022,1,-0.652196,0.584714,0,0,-0.556841,-0.341702,0.254747,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25835,1096,2022,22,0.144304,0.373129,0,0,-0.556841,-0.341702,-0.956486,...,0,0,0,0,0,0,0,0,0,0
25836,1096,2022,22,0.781503,0.377508,0,0,-0.556841,-0.341702,0.456619,...,0,0,0,0,0,0,0,0,0,0
25837,1096,2022,22,-0.970795,-1.618941,1,0,1.852542,-0.341702,1.869724,...,0,0,0,0,0,0,0,0,0,0
25838,1096,2022,22,1.418703,-1.618941,1,0,-0.556841,-0.341702,-0.148997,...,0,0,0,0,0,0,0,0,0,0


# Grid Search to find best network Parameters


This process kept on crashing due to high number of features previously so had to perform feature engieering carefully to prevent it from crashing .
Even thought of doing random search instead of grid search but random search is less effective than grid search it might spoil the model performance so went with grid search.

The code is commented because it takes 4hr 45mins  to run. The Results are given in a seperate CSV file with the project

In [105]:
# Define features and target
#X_train = df_train.drop(columns=['IsRaceWinner'])  # Features (excluding target variable)
#y_train = df_train['IsRaceWinner']  # Target variable

#if len(X_train.shape) > 2:
#    X_train = X_train.reshape(X_train.shape[0], -1)
#print("Reshaped X_train:", X_train.shape)

# Ensure y_train is 1D
#y_train = y_train.values.flatten()

# Split the training data into train and validation sets (80% train, 20% validation)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# setup for hyperparameter tuning
#neurons_layer1 = [10, 25, 50, 100]
#neurons_layer2 = [0, 10, 25, 50, 100]
#neurons_layer3 = [0, 10, 25, 50, 100]
#epochs = [20, 50, 100]

# Placeholder for storing results
#results = []

# Features after one hot encoding
#num_features = X_train.shape[1]

# Loop through all hyperparameter combinations for tuning
#for e in tqdm(epochs):
#    for n1 in neurons_layer1:
#       for n2 in neurons_layer2:
#           for n3 in neurons_layer3:
#
               # Initialize a model for this configuration
#                model = tf.keras.Sequential()

                # Input layer
#                model.add(tf.keras.layers.InputLayer(input_shape=(num_features,)))
#                model.add(tf.keras.layers.Dense(255, activation='relu'))  # Initial dense layer

                # Hidden Layer 1
#                model.add(tf.keras.layers.Dense(n1, activation='relu'))

                # Optional hidden layers
#                if n2 != 0:
#                    model.add(tf.keras.layers.Dense(n2, activation='relu'))
#                if n3 != 0:
#                    model.add(tf.keras.layers.Dense(n3, activation='relu'))

                # Output Layer
#                model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

                # Compile the model
#                model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

                # Fit the model (train it on your data)
#                history = model.fit(X_train, y_train, epochs=e, batch_size=32, validation_data=(X_val, y_val), verbose=0)

                # Evaluate the model's performance
#                y_pred = model.predict(X_val)
#                y_pred = (y_pred > 0.5).astype(int)  # Convert probabilities to binary class predictions
#                accuracy = np.mean(y_pred == y_val)

                # Store the results
#                results.append([e, n1, n2, n3, accuracy])

# Convert results into a DataFrame
#results_df = pd.DataFrame(results, columns=['Epochs', 'Neurons_Layer1', 'Neurons_Layer2', 'Neurons_Layer3', 'Accuracy'])

# best_config = results_df.loc[results_df['Accuracy'].idxmax()]
# print("Best Hyperparameters:", best_config)

# GRID SEARCH OUTPUT

The results dataframe is coverted to csv file and is submitted along with the project as a seperate csv

In [106]:
# Grid Search output csv
#results_df.to_csv('grid_search.csv', index=False)

# Model Predictions

In [111]:
# Define features and target from df_test
X_test = df_test.drop(columns=['IsRaceWinner'])  # Features (excluding target variable)
y_test = df_test['IsRaceWinner']  # Target variable

if len(X_test.shape) > 2:
    X_test = X_test.reshape(X_test.shape[0], -1)

y_test = y_test.values.flatten()

# Split the data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_test, y_test, test_size=0.2, random_state=42)

# The best hyperparameters from the grid search we found above
epochs = 20
neurons_layer1 = 25
neurons_layer2 = 10
neurons_layer3 = 0
num_features = X_train.shape[1]

# Define the class weights to handle class imbalance
class_weights = {0: 1, 1: 4.6}

# Initialize a model with the best configuration
model = tf.keras.Sequential()

# Input layer (use num_features as the input shape)
model.add(tf.keras.layers.InputLayer(input_shape=(num_features,)))
model.add(tf.keras.layers.Dense(255, activation='relu'))

# Hidden Layer 1
model.add(tf.keras.layers.Dense(neurons_layer1, activation='relu'))

# Optional hidden layers based on configuration
if neurons_layer2 != 0:
    model.add(tf.keras.layers.Dense(neurons_layer2, activation='relu'))
if neurons_layer3 != 0:
    model.add(tf.keras.layers.Dense(neurons_layer3, activation='relu'))

# Output Layer (Binary classification)
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit the model with class weights we set few lines above
history = model.fit(X_train, y_train, epochs=epochs, batch_size=32, validation_data=(X_val, y_val), class_weight=class_weights, verbose=1)

# Evaluate the model's performance on the validation set
y_pred_proba = model.predict(X_val)
y_pred = (y_pred_proba > 0.5).astype(int)

# Now, make predictions on the test set (df_test)
y_test_pred_proba = model.predict(X_test)
y_test_pred = (y_test_pred_proba > 0.5).astype(int)

Epoch 1/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.7678 - loss: 7.9088 - val_accuracy: 0.9659 - val_loss: 0.3183
Epoch 2/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7868 - loss: 1.6186 - val_accuracy: 0.9659 - val_loss: 0.1400
Epoch 3/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7371 - loss: 0.9177 - val_accuracy: 0.9659 - val_loss: 0.3626
Epoch 4/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9642 - loss: 0.4365 - val_accuracy: 0.9659 - val_loss: 0.2404
Epoch 5/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9578 - loss: 0.4550 - val_accuracy: 0.9659 - val_loss: 0.1666
Epoch 6/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8963 - loss: 0.7352 - val_accuracy: 0.9659 - val_loss: 0.2339
Epoch 7/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9629 - loss: 0.3266 - val_accuracy: 0.9659 - val_loss: 0.0924
Epoch 8/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9448 - loss: 0.3947 - val_accuracy: 0.9659 - val_los

In [112]:
# Classification Report for Validation Set
print("Classification Report (Validation Set):\n", classification_report(y_val, y_pred))
print("Confusion Matrix (Validation Set):\n", confusion_matrix(y_val, y_pred))

Classification Report (Validation Set):
               precision    recall  f1-score   support

           0       0.98      0.99      0.98        85
           1       0.50      0.33      0.40         3

    accuracy                           0.97        88
   macro avg       0.74      0.66      0.69        88
weighted avg       0.96      0.97      0.96        88

Confusion Matrix (Validation Set):
 [[84  1]
 [ 2  1]]


In [113]:
# Classification Report for Test set
print("Classification Report (Test Set):\n", classification_report(y_test, y_test_pred))
print("Confusion Matrix (Test Set):\n", confusion_matrix(y_test, y_test_pred))

Classification Report (Test Set):
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       418
           1       0.76      0.73      0.74        22

    accuracy                           0.97       440
   macro avg       0.87      0.86      0.87       440
weighted avg       0.97      0.97      0.97       440

Confusion Matrix (Test Set):
 [[413   5]
 [  6  16]]


## Append the predictions with df_test3 we created before encoding to see the results

Looking at this we can see that out of 22 races our model predicted 16 races correctly. There were some races where the model predicted two winners and the first race of the season our model predicted no winner which is a downside to our model. This can be improved further doing optimizations in grid search, new feature engineering, adding more columns related to weather , etc. to improve accuracy of the model even more . After all its hard to predict F1 races since they depend on various factors both present and past.

In [116]:
# Add the predicted values as a new column to the original df_test DataFrame
df_eval['PredictedIsRaceWinner'] = y_test_pred
# Now you can see the original data along with the predictions
df_eval[['Driver_name','Circuit','IsRaceWinner', 'PredictedIsRaceWinner']]  # Display the first few rows of original and predicted values

,Driver_name,Circuit,IsRaceWinner,PredictedIsRaceWinner
24980,Charles Leclerc,Bahrain Grand Prix,1,0
24981,Carlos Sainz,Bahrain Grand Prix,0,0
24982,Lewis Hamilton,Bahrain Grand Prix,0,0
24983,George Russell,Bahrain Grand Prix,0,0
24984,Kevin Magnussen,Bahrain Grand Prix,0,0
...,...,...,...,...
25835,Mick Schumacher,Abu Dhabi Grand Prix,0,0
25836,Kevin Magnussen,Abu Dhabi Grand Prix,0,0
25837,Lewis Hamilton,Abu Dhabi Grand Prix,0,0
25838,Nicholas Latifi,Abu Dhabi Grand Prix,0,0
